## Assignment 3: Evaluation of Deep Learning Models on Image Classification

### CS 597: Special Topics on Deep Learning

### Adam Torek

### Professor: Jun Zhuang

### Spring 2025 Semester

### Library Imports/Setup

<hr>

In [2]:
import torch
from torchvision.models import resnet50, swin_b, mobilenet_v3_small
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import RandomHorizontalFlip, Normalize, ToTensor, Compose, RandomRotation
import tqdm

In [3]:
batch_size = 32
num_epochs = 10
learning_rate = 1e-4

### Dataset Construction

<hr>

In [4]:
cifar_dataset = CIFAR10(download=True, root="data")

Files already downloaded and verified


In [5]:
class CIFARSubsetDataSet(Dataset):
    def __init__(self, dataset_tuples, transform=None):
        self.dataset_tuples = dataset_tuples
        self.transform = transform
        self.dataset_len = len(dataset_tuples)
        self.tensor_transform = ToTensor()

    def __len__(self):
        return self.dataset_len
    
    def __getitem__(self, idx):
        dataset_item = self.dataset_tuples[idx]
        image = self.tensor_transform(dataset_item[0])
        
        labels = [0,0]
        labels[dataset_item[1]] = 1
        if self.transform:
            image = self.transform(image)
        
        return {"image": image, "label": torch.tensor(labels, dtype=torch.float)}  

In [6]:
cifar_data_subset = []
cifar_extraction_dataloader = iter(cifar_dataset)

for image, label in cifar_extraction_dataloader:
    if label == 0 or label == 1:
        cifar_data_subset.append((image, label))

In [7]:
augmentation_transforms = Compose([RandomHorizontalFlip(), RandomRotation(degrees=0.02)])

cifar_binary_dataset = CIFARSubsetDataSet(dataset_tuples=cifar_data_subset, transform=augmentation_transforms)

In [8]:
cifar_binary_dataloader = DataLoader(cifar_binary_dataset, batch_size=batch_size, shuffle=False)

In [14]:
def train_model(model, num_epochs, learning_rate, dataloader, loss_func):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    training_loop = tqdm.tqdm(dataloader, leave=True)
    for epoch in range(num_epochs):
        total_batch_steps = 0
        total_batch_loss = 0
        for batch in training_loop:
            optimizer.zero_grad()
            images = batch["image"]
            labels = batch["label"]
            outputs = model(images)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()
            total_batch_steps += len(images)
            total_batch_loss += loss.item()

            #training_loop.postfix(loss.item())

        print("The average training loss after epoch {0} is {1}".format(str(epoch), float(total_batch_loss/total_batch_steps)))

    return model

In [15]:
mobilenet_model = mobilenet_v3_small(weights=None)
mobilenet_model.train()
mobilenet_model.classifier[3] = torch.nn.Linear(in_features=mobilenet_model.classifier[3].in_features, out_features=2)
train_model(mobilenet_model, num_epochs=num_epochs, learning_rate=learning_rate, dataloader=cifar_binary_dataloader, loss_func=torch.nn.BCEWithLogitsLoss())

100%|██████████| 313/313 [01:31<00:00,  3.43it/s]


The average training loss after epoch 0 is 0.015999638637900354
The average training loss after epoch 1 is 0.01265767001658678


KeyboardInterrupt: 

In [132]:
for classifier in mobilenet_model.classifier:
    print(type(classifier))

<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.Hardswish'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.linear.Linear'>
